# EEG Batch Import

This notebook batch imports all the pairs of `xdf` and `json` files in the Data folder. Note that one must first create the `json` files by running the [eeg_processing](eeg_preprocessing.ipynb) notebook first.

## Import libraries

In [18]:
# Standard libraries
import mne
import json
import numpy as np
from mne.preprocessing import ICA

# Custom libraries
from Functions import import_data
from Functions import data_tools

## Select files to import

Add the name of the files to import. Note that each file in the list must have a `xdf` and `json` file with that name.

In [19]:
# Settings
# - List of files to import and pre-process
files_to_import = [
    "sub-P004_ses-S001_task-T1_run-001_eeg",
]

save_npy = True    # Boolean to save numpy epochs 

## Import and preprocess data



In [20]:
# Preallocate variables 
eeg_mne = [None] * len(files_to_import) # List of MNE objects with raw data

for file in files_to_import:
    # Import settings saved in json
    with open(f"Data\{file}.json", "r") as file_object:
        settings = json.load(file_object)

    # Import EEG data
    [eeg_ts, eeg_data, eeg_fs] = import_data.read_xdf(
        f"Data\{file}.xdf",
        picks=settings["ch_names"]
        )
    
    # Create MNE array
    info = mne.create_info(settings["ch_names"], eeg_fs, ch_types='eeg')    # Create info properties
    mne_raw = mne.io.RawArray(eeg_data, info=info)                          # Create MNE raw array
    mne_raw.set_montage('standard_1020')                                    # Set montage to array

    # Apply band-pass filter
    filt_raw = mne_raw.copy().filter(
        l_freq = settings["lpf_fc"],
        h_freq = settings["hpf_fc"],
        picks = settings["ch_names"]
        )
    
    # Create ICA
    ica = ICA(
        n_components = settings["n_components"],
        max_iter = settings["max_iter"],
        random_state = settings["random_state"]
        )
    ica.fit(filt_raw, picks=settings["ch_names"])

    # Remove ICA components
    ica.exclude = settings["ica_exclude"]
    ica.apply(filt_raw)

    # Epoch data
    [marker_ts, markers] = import_data.read_xdf_unity_markers(f"Data\{file}.xdf")
    [eeg_epochs, epoch_labels] = data_tools.epochs_from_unity_markers(
        eeg_time = eeg_ts,
        eeg_data = filt_raw.get_data(),
        marker_time = marker_ts,
        marker_data = markers
        )
    
    # - Get stimuli and frequencies labels
    fixed_labels = data_tools.fix_labels(epoch_labels)          # Fix labels that might be incorrect in pilot trials
    dict_of_stimuli = data_tools.get_tvep_stimuli(fixed_labels) # Dictionary of unique stimulus
    dict_of_freqs = {0:"2", 1:"9.6", 2:"16", 3:"36"}            # Dictionary of stimulus frequencies

    # - Create array of eeg epochs organized as [stimuli, freq, chans, samples]
    eeg_epochs_organized = data_tools.epochs_stim_freq(
        eeg_epochs = eeg_epochs,
        labels = fixed_labels,
        stimuli = dict_of_stimuli,
        freqs = dict_of_freqs
        )
    
    # Save data to NPY file
    if (save_npy):
        np.save(
            f"Data\{file}-batch.npy",
            eeg_epochs_organized,
            )   

Creating RawArray with float64 data, n_channels=16, n_times=1007744
    Range : 0 ... 1007743 =      0.000 ...  3936.496 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (6.605 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Fitting ICA to data using 16 channels (please be patient, this may take a while)


[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.3s finished
C:\Users\danie\AppData\Local\Temp\ipykernel_36568\3308386101.py:33: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(filt_raw, picks=settings["ch_names"])


Selecting by number: 16 components
Fitting ICA took 10.0s.
Applying ICA to Raw instance
    Transforming to ICA space (16 components)
    Zeroing out 1 ICA component
    Projecting back using 16 PCA components
